# Arguments

In [1]:

warmup_model_path = 'vgg-warmup-model.h5'
model_save_path = 'vgg16-fined-tuned.h5'
dataset_name = 'plantVillage-tomato-mohanty'
dataset_path = '../datasets/' + dataset_name
checkpoint_path = 'finetuning-checkpoints.hdf5'
input_width = 224
input_height = 224
input_depth = 3

batch_size = 32
num_of_epochs = 100

# Get classes
import os
import re
classes = os.listdir(dataset_path)
class_names = []

for i in classes:
    if(re.search("Tomato___", i)):
        class_names.append(i)
    
print('Classes: ', class_names)
print(len(class_names))


Classes:  ['Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Tomato_mosaic_virus', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Bacterial_spot', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___healthy', 'Tomato___Septoria_leaf_spot']
10


# Load model from disk

In [2]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)





Using TensorFlow backend.


# Print index of layers

In [3]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {}\t{}".format(i,layer.__class__.__name__))

[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D
[INFO] 19	Flatten
[INFO] 20	Dense
[INFO] 21	Dropout
[INFO] 22	Dense


# Unfreeze final CONV layers

In [4]:
for layer in model.layers[15:]:
    layer.trainable = True

# Compile model

In [5]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.001)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Load data

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

# Load image paths
print("[INFO] loading images...")
image_paths = list(paths.list_images(dataset_path))

# Initial image preprocessing
aap = AspectAwarePreprocessor(input_width, input_height)
iap= ImageToArrayPreprocessor()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(image_paths,verbose=500)
data = data.astype("float") / 255.0


# train test split
(train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

# convert the labels from integers to vectors
train_y = LabelBinarizer().fit_transform(train_y)
test_y = LabelBinarizer().fit_transform(test_y)

[INFO] loading images...
[INFO]: Processed 500/18160
[INFO]: Processed 1000/18160
[INFO]: Processed 1500/18160
[INFO]: Processed 2000/18160
[INFO]: Processed 2500/18160
[INFO]: Processed 3000/18160
[INFO]: Processed 3500/18160
[INFO]: Processed 4000/18160
[INFO]: Processed 4500/18160
[INFO]: Processed 5000/18160
[INFO]: Processed 5500/18160
[INFO]: Processed 6000/18160
[INFO]: Processed 6500/18160
[INFO]: Processed 7000/18160
[INFO]: Processed 7500/18160
[INFO]: Processed 8000/18160
[INFO]: Processed 8500/18160
[INFO]: Processed 9000/18160
[INFO]: Processed 9500/18160
[INFO]: Processed 10000/18160
[INFO]: Processed 10500/18160
[INFO]: Processed 11000/18160
[INFO]: Processed 11500/18160
[INFO]: Processed 12000/18160
[INFO]: Processed 12500/18160
[INFO]: Processed 13000/18160
[INFO]: Processed 13500/18160
[INFO]: Processed 14000/18160
[INFO]: Processed 14500/18160
[INFO]: Processed 15000/18160
[INFO]: Processed 15500/18160
[INFO]: Processed 16000/18160
[INFO]: Processed 16500/18160
[INFO

# Checkpoints

In [7]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]

#load checkpoints if existing
import os

epochs_done = 1

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)
    num_of_epochs = num_of_epochs - epochs_done

# Fit model

In [8]:
model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

model.save(model_save_path)

Epoch 1/100
425/425 [==============================] - 220s 518ms/step - loss: 2.2531 - accuracy: 0.1814 - val_loss: 2.0092 - val_accuracy: 0.3134

Epoch 00001: val_loss improved from inf to 2.00916, saving model to finetuning-checkpoints.hdf5
Epoch 2/100
425/425 [==============================] - 216s 509ms/step - loss: 1.7757 - accuracy: 0.3650 - val_loss: 1.6072 - val_accuracy: 0.4692

Epoch 00003: val_loss improved from 1.69174 to 1.60718, saving model to finetuning-checkpoints.hdf5
Epoch 4/100
425/425 [==============================] - 215s 506ms/step - loss: 1.6862 - accuracy: 0.3873 - val_loss: 1.5281 - val_accuracy: 0.4855

Epoch 00004: val_loss improved from 1.60718 to 1.52807, saving model to finetuning-checkpoints.hdf5
Epoch 5/100
425/425 [==============================] - 215s 505ms/step - loss: 1.6237 - accuracy: 0.4005 - val_loss: 1.4281 - val_accuracy: 0.4912

Epoch 00005: val_loss improved from 1.52807 to 1.42805, saving model to finetuning-checkpoints.hdf5
Epoch 6/100


# Evaluate

In [9]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)
print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=class_names))

[INFO] evaluating after initialization...
                                               precision    recall  f1-score   support

Tomato___Spider_mites Two-spotted_spider_mite       0.99      0.97      0.98       532
                 Tomato___Tomato_mosaic_virus       0.98      0.67      0.79       259
                        Tomato___Early_blight       0.96      0.97      0.96       476
                         Tomato___Late_blight       0.98      0.91      0.94       239
                           Tomato___Leaf_Mold       0.89      1.00      0.94       435
                      Tomato___Bacterial_spot       0.92      0.98      0.95       414
                         Tomato___Target_Spot       0.86      0.95      0.90       369
       Tomato___Tomato_Yellow_Leaf_Curl_Virus       1.00      0.98      0.99      1345
                             Tomato___healthy       0.94      0.94      0.94        78
                  Tomato___Septoria_leaf_spot       0.98      0.99      0.99       393
